# A simple implementation on one data point

## Set up

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/mcts-for-llm/')

In [4]:
%pwd

'/content/drive/MyDrive/mcts-for-llm'

In [5]:
!pip install gymnasium accelerate transformers databench-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
import transformers
from dyna_gym.pipelines import uct_for_hf_transformer_pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import subprocess
import shlex
import zipfile
import torch
from databench_eval import Runner, Evaluator, utils
from datasets import load_dataset
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from databench_eval import Runner, Evaluator, utils
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## Load data

In [7]:
semeval_dev_qa = load_dataset("cardiffnlp/databench", name="semeval", split="dev")
ds_id = "050_ING"
df_all = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{ds_id}/all.parquet")
qa = semeval_dev_qa.select([0])
gold_ans = qa[0]['sample_answer']
print(qa)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/43.6k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

qa.parquet:   0%|          | 0.00/6.58k [00:00<?, ?B/s]

...

qa.parquet:   0%|          | 0.00/6.89k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/7.19k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/988 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'type', 'columns_used', 'column_types', 'sample_answer', 'dataset'],
    num_rows: 1
})


In [103]:
model_name = "stabilityai/stable-code-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")
vocab_size = tokenizer.vocab_size

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [115]:
horizon = 250  # maximum number of steps / tokens to generate in each episode

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## generate prompt, code execution, evaluation

In [114]:
def call_model(prompts):
    """
    tokenize prompt, model generate;
    prompts: str
    """
    # print("*" * 50)
    inputs = tokenizer(prompts, return_tensors="pt").to(model.device)
    tokens = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(tokens[0], skip_special_tokens=True)
    print(result)
    return result


def prompt_generator(row):
    question = row['question']
    df = df_all
    prompt = f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: {question}
def answer(df: pd.DataFrame) -> {row["type"]}:
    df.columns = {list(df.columns)}
    return"""
    return prompt


def post_process(response: str, dataset: str):
    """
    process and execute the code generated by model
    response: the textual completion generated by the model
    dataset: the dataset we want to query, in this one data point case is "050_ING" which is df_all(I load it locally) above;
    """

    """
    Below is how this process actually look like
    df = pd.DataFrame({1:"aa",2:"bb"})
    response = "def print():\n return print(hello world)"
    global ans
    def answer(df):
        return response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")
    ans = answer(df)  #ans is the predicted textual answer of the question by model, which should match with the gold answer;
    """

    try:
        # df = loader(dataset)
        df = dataset
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )
        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"


def compare(value, truth):
    return str(value).strip == str(truth).strip()

In [84]:
prompt = prompt_generator(qa[0])
completion = call_model(prompt)


# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: Is the most favorited author mainly communicating in Spanish?
def answer(df: pd.DataFrame) -> boolean:
    df.columns = ['id', 'author_id', 'author_name', 'author_handler', 'author_avatar', 'lang', 'type', 'text', 'date','mention_ids','mention_names','retweets', 'favorites', 'links', 'links_first', 'image_links', 'image_links_first', 'rp_user_id', 'rp_user_name', 'location', 'tweet_link','search']
    return df.loc[df['lang'] == 'es'].favorites.sum() > df.loc[df['lang']!= 'es'].favorites.sum()

# TODO: complete the following function in one line. It should give the answer to: What is the average number of favorites per tweet?
def answer(df: pd.DataFrame) -> float:
    df.columns = ['id', 'author_id

In [85]:
def massage_response(response:str):
  return response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

In [86]:
print(completion)
print("*"*20)
print(post_process(completion, df_all))


# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: Is the most favorited author mainly communicating in Spanish?
def answer(df: pd.DataFrame) -> boolean:
    df.columns = ['id', 'author_id', 'author_name', 'author_handler', 'author_avatar', 'lang', 'type', 'text', 'date','mention_ids','mention_names','retweets', 'favorites', 'links', 'links_first', 'image_links', 'image_links_first', 'rp_user_id', 'rp_user_name', 'location', 'tweet_link','search']
    return df.loc[df['lang'] == 'es'].favorites.sum() > df.loc[df['lang']!= 'es'].favorites.sum()

# TODO: complete the following function in one line. It should give the answer to: What is the average number of favorites per tweet?
def answer(df: pd.DataFrame) -> float:
    df.columns = ['id', 'author_id

In [108]:
massage_response(completion)

"df.loc[df['lang'] == 'es'].favorites.sum() > df.loc[df['lang']!= 'es'].favorites.sum()"

## RL args:
### reward function is the execution of the code

In [128]:
def reward_function(sentence):
    """
    if (get correct NL answer) -> 1
    else -> 0
    """
    if compare(sentence,qa[0]['answer']):
        return 1
    else:
        return 0


# arguments for the UCT agent
uct_args = dict(
    rollouts=3,
    gamma=1.,
    width=5,
    alg='uct',  # or p_uct
)

# will be passed to huggingface model.generate()
model_generation_args = dict(
    top_k=3,
    top_p=0.9,
    do_sample=True,
    temperature=0.2,
)

In [129]:
pipeline = uct_for_hf_transformer_pipeline(
    model=model,
    tokenizer=tokenizer,
    horizon=horizon,
    reward_func=reward_function,
    uct_args=uct_args,
    model_generation_args=model_generation_args,
    should_plot_tree=True,  # plot the tree after generation
)

In [130]:
input_str = prompt_generator(qa[0])
input_str

'\n# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?\ndef example(df: pd.DataFrame) -> int:\n    df.columns=["A"]\n    return df.shape[0]\n\n# TODO: complete the following function in one line. It should give the answer to: Is the most favorited author mainly communicating in Spanish?\ndef answer(df: pd.DataFrame) -> boolean:\n    df.columns = [\'id\', \'author_id\', \'author_name\', \'author_handler\', \'author_avatar\', \'lang\', \'type\', \'text\', \'date\', \'mention_ids\', \'mention_names\', \'retweets\', \'favorites\', \'links\', \'links_first\', \'image_links\', \'image_links_first\', \'rp_user_id\', \'rp_user_name\', \'location\', \'tweet_link\', \'search\']\n    return'

In [131]:
outputs = pipeline(input_str=input_str)

Rolling out:  67%|██████▋   | 2/3 [03:34<01:46, 106.84s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
                                                           

 ' df' prob 0.6328125 returns [0.0]
	 '.' prob 0.400390625 returns []
	 "['" prob 0.2431640625 returns []
	 '[' prob 0.189453125 returns []
 ' True' prob 0.09716796875 returns [0.0]
	 ' if' prob 0.5 returns []
	 '\n' prob 0.34375 returns []
	 '\n\n' prob 0.07666015625 returns []
 ' (' prob 0.058837890625 returns [0.0]
	 'df' prob 0.8984375 returns []
	 'True' prob 0.0185546875 returns []
Tree plotted and saved to tree-20241216-102443.pdf


In [132]:
for text, reward in zip(outputs['texts'], outputs['rewards']):
    print("==== Text ====")
    print(text)
    print("==== Reward:", reward, "====")
    print()


==== Text ====

# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: Is the most favorited author mainly communicating in Spanish?
def answer(df: pd.DataFrame) -> boolean:
    df.columns = ['id', 'author_id', 'author_name', 'author_handler', 'author_avatar', 'lang', 'type', 'text', 'date','mention_ids','mention_names','retweets', 'favorites', 'links', 'links_first', 'image_links', 'image_links_first', 'rp_user_id', 'rp_user_name', 'location', 'tweet_link','search']
    return df.lang.value_counts()[0] > df.lang.value_counts()[1]

# TODO: complete the following
==== Reward: 0 ====

==== Text ====

# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -